In [ ]:
# !pip install deep-sort-realtime

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


In [15]:
import cv2
import numpy as np
from ultralytics import YOLO
from sort import Sort

# Load YOLOv8 face detection model
model = YOLO('yolov8n-face.pt')

# Initialize SORT tracker
tracker = Sort()

# Initialize ID mapping and next ID counter
id_mapping = {}
next_id = 1

# Open video file or webcam
video_path = 'test_video.mp4'  # or use 0 for webcam
cap = cv2.VideoCapture(video_path)

# Get video writer ready to save output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
output_path = 'output_4.mp4'
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Set confidence threshold
confidence_threshold = 0.56

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference on the frame
    results = model(frame)

    # Extract bounding boxes and confidence scores
    detections = []
    boxes = results[0].boxes  # YOLOv8 bounding boxes

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordinates
        conf = box.conf[0]  # Confidence
        cls = int(box.cls[0])  # Class (0 = face)
        
        # Apply confidence threshold
        if cls == 0 and conf >= confidence_threshold:  # Class 0 is face in the custom model
            detections.append([x1, y1, x2, y2, conf])
            print(f"Detected face: ({x1}, {y1}, {x2}, {y2}) with confidence: {conf:.2f}")

    # Update tracker with detections
    if len(detections) > 0:
        trackers = tracker.update(np.array(detections))
        
        # Assign sequential IDs based on tracked faces
        for track in trackers:
            track_id = int(track[4])  # Existing SORT ID
            print(f"Tracked ID from SORT: {track_id}")
            
            # If the track_id is not in the ID mapping, assign a new unique ID
            if track_id not in id_mapping:
                id_mapping[track_id] = next_id
                next_id += 1
            
            unique_id = id_mapping[track_id]
            print(f"Assigned Unique ID: {unique_id}")

            # Draw bounding boxes and tracker IDs
            x1, y1, x2, y2 = map(int, track[:4])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {unique_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Save the frame with bounding boxes to the output video
    out.write(frame)

    # Display the frame (optional)
    cv2.imshow('YOLOv8 Face Detection with Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture, video writer, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()

# Print number of unique faces detected
print(f'Number of unique faces detected: {len(id_mapping)}')


Ultralytics YOLOv8.0.17  Python-3.11.7 torch-2.4.1+cpu CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Detected face: (642, 0, 1278, 694) with confidence: 0.59
Tracked ID from SORT: 1657
Assigned Unique ID: 1
Detected face: (497, 3, 695, 107) with confidence: 0.57
Tracked ID from SORT: 1658
Assigned Unique ID: 2
Detected face: (494, 5, 677, 151) with confidence: 0.71
Tracked ID from SORT: 1658
Assigned Unique ID: 2
Detected face: (490, 5, 677, 187) with confidence: 0.68
Detected face: (477, 7, 671, 222) with confidence: 0.76
Tracked ID from SORT: 1658
Assigned Unique ID: 2
Detected face: (478, 8, 667, 257) with confidence: 0.73
Tracked ID from SORT: 1658
Assigned Unique ID: 2
Detected face: (472, 27, 663, 287) with confidence: 0.76
Tracked ID from SORT: 1658
Assigned Unique ID: 2
Detected face: (473, 35, 662, 296) with confidence: 0.76
Tracked ID from SORT: 1658
Assigned Unique ID: 2
Detected face: (478, 38, 663, 297) with confidence: 0.80
Tracked ID from SORT: 1658
Assigned Unique ID: 2
Detected face: (489, 38, 670, 295) with confidence: 0.75
Tracked ID from SORT: 1658
Assigned Unique 